# Daily Financial News for 6000+ Stocks
### Sentimentanalysis-Modelling of Headlines for Stocks and there prices
@author DHR <br>

## Requirements

In [35]:
#immport basic DataScience Modules
#!! for pip-modul-list to install @see requirements.txt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [36]:
# import final DataSet for Modelling
data = pd.read_csv('../data/analyst_ratings_processed_final.csv')

## Sentiment and Stock Increase as Binary
positive Sentiment = 1;
neutral Sentiment = 0;
negative Sentiment = -1

Stock Price Increase by more than x = 1;
Stock Price doesnt change more than x = 0;
Stock Price Decrease by more than x = -1

In [37]:
def stockDevelopment(row):
    # stock_differenz = row['open'] - row['close']
    quotient = row['close']/row['open']
    if quotient < 0.9:
        return -1
    elif quotient > 1.1:
        return 1
    else: return 0

In [38]:
data["stockPrice_Change"] = data.apply(stockDevelopment, axis=1)

In [39]:
def getBinarySenti(row):
    if row["senti_score"] < 0:
        return -1
    elif row['senti_score'] > 0:
        return 1
    else: return 0

In [40]:
data['senti_Binary'] = data.apply(getBinarySenti, axis=1)

### Check relation between Stockprice increase and Sentiment

In [41]:
def checkCorrelation_pos(row):
    if row['stockPrice_Change'] == row["senti_Binary"]:
        return 1
    else: return 0

In [42]:
data["correlation_pos"] = data.apply(checkCorrelation_pos, axis=1)

In [43]:
def checkCorrelation_neg(row):
    if ((row['stockPrice_Change'] != row["senti_Binary"]) and (row['stockPrice_Change'] != 0) and (row["senti_Binary"] !=0)):
        return 1
    else: return 0

In [44]:
data["correlation_neg"] = data.apply(checkCorrelation_neg, axis=1)

In [45]:
numberRowsCorrect = data[data["correlation_pos"] == 1].shape[0]
numberRows = data.shape[0]
accuracy = numberRowsCorrect / numberRows
print("Von " + str(numberRows) + " Headlines mit Sentiment-Score korrelieren " + str(numberRowsCorrect) +" positiv mit einem Aktienanstieg.\nDas ergbit eine Accuracy von: " + str(accuracy))

Von 161478 Headlines mit Sentiment-Score korrelieren 61517 positiv mit einem Aktienanstieg.
Das ergbit eine Accuracy von: 0.3809621124859114


In [46]:
numberRowsIncorrect = data[data["correlation_neg"] == 1].shape[0]
numberRows = data.shape[0]
accuracy = numberRowsIncorrect / numberRows
print("Von " + str(numberRows) + " Headlines mit Sentiment-Score korrelieren " + str(numberRowsIncorrect) +" negativ mit einem Aktienanstieg.\nDas ergbit eine Accuracy von: " + str(accuracy))

Von 161478 Headlines mit Sentiment-Score korrelieren 3520 negativ mit einem Aktienanstieg.
Das ergbit eine Accuracy von: 0.02179863510818811
